# 写真に映る動物が犬か猫かを分類する

In [18]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.svm import SVC
from sklearn import metrics

In [19]:
# 訓練データ用CSVの読込
train_data = pd.read_csv("dc_photos/train/train_data.csv")

In [20]:
# テストデータ用CSVの読込
test_data = pd.read_csv("dc_photos/test/test_data.csv")

## ・データの前処理や水増し

In [21]:
# 訓練用写真の読込

# データ保管する領域の確保
train_len = len(train_data)
# 左右に回転させたものを用意するため、2倍の容量を確保
x_train = np.empty((train_len * 2, 5625), dtype=np.uint8)
y_train = np.empty(train_len * 2, dtype=np.uint8)

# 繰り返し
for i in range(len(train_data)):
    name        = train_data.loc[i, "File name"]
    train_img   = Image.open(f"dc_photos/train/{name}.jpg").convert("L")
    train_img   = np.array(train_img)
    train_img_f = train_img.flatten()
    x_train[i]  = train_img_f
    y_train[i]  = train_data.loc[i, "DC"]
    
    # 左右反転
    train_img_lr           = np.fliplr(train_img)
    train_img_lr_f         = train_img_lr.flatten()
    x_train[i + train_len] = train_img_lr_f
    y_train[i + train_len] = train_data.loc[i, "DC"]

In [22]:
# テスト用写真読み込み
# データを保管する領域の確保
test_len = len(test_data)
x_test   = np.empty((test_len, 5625), dtype=np.uint8)
y_test   = np.empty(test_len, dtype=np.uint8)

# 繰り返し
for i in range(test_len):
    name = test_data.loc[i, "File name"]
    test_img = Image.open(f"dc_photos/test/{name}.jpg").convert("L")
    test_img = np.array(test_img)
    test_img_f = test_img.flatten()
    x_test[i] = test_img_f
    y_test[i] = test_data.loc[i, "DC"]

In [23]:
# SVCの分類モデルの作成
dog_cat = SVC(kernel = "linear")
# 訓練データを分類モデルに設定する
dog_cat.fit(x_train, y_train)

SVC(kernel='linear')

In [24]:
# テストデータを使って分類を実行する
y_pred = dog_cat.predict(x_test)

In [25]:
# 予測値を表示する
print(y_pred)

[0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 1
 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [26]:
# 実際の値を表示する
print(y_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [27]:
# 混同行列で集計結果を表示する
print(metrics.confusion_matrix(y_test, y_pred))

[[36 14]
 [ 5 45]]


In [28]:
# 正答率を表示する
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.72      0.79        50
           1       0.76      0.90      0.83        50

    accuracy                           0.81       100
   macro avg       0.82      0.81      0.81       100
weighted avg       0.82      0.81      0.81       100

